# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [ ]:
# Import Python packages 
import pandas as pd
import cassandra
from cassandra.query import BatchStatement
import re
import os
import glob
import numpy as np
import json
import csv
import time
from contextlib import contextmanager

#### Define helper functions

In [ ]:
def pandas_factory(colnames, rows):
    """
    tuple factory to create pandas data frame from cassandra row data
    """
    return pd.DataFrame(rows, columns=colnames)

@contextmanager
def measure_time(message='Finished in {}s.'):
    """
    Description: messures execution time of decorated code

    Arguments:
        message: message to be shown at the end of execution. Use {} to display time in seconds.
 
    Returns:
        None
    """
    start = time.time()

    yield

    elapsed = round(time.time() - start, 2)
    print(message.format(elapsed))

#### Creating list of filepaths to process original event csv data files

In [ ]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#print(file_path_list)

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [ ]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
# extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [ ]:
# check the number of rows in the csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [ ]:
# Make a connection to a Cassandra instance 
# (127.0.0.1)
db_host = os.getenv('DB_HOST', 'localhost')
# print(db_host)

from cassandra.cluster import Cluster
cluster = Cluster([db_host])

# To establish connection and begin executing queries, need a session
session = cluster.connect()
session.row_factory = pandas_factory

#### Create Keyspace

In [ ]:
# Create a Keyspace 
session.execute("""
        CREATE KEYSPACE IF NOT EXISTS udacity
        WITH REPLICATION = 
        { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

#### Set Keyspace

In [ ]:
# Set KEYSPACE to the keyspace specified above
session.set_keyspace("udacity")

## Query 1:  
## Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

SessionId only is not enough to uniquely identify each row. So we use the combination of sesseionId and itemInSession as primary key. On the other hand there will be not too much data returned by using only sessionId in the where-clause, so we use only sessionId as partition key.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS song_info_by_session_and_item (
                sessionId int,
                itemInSession int,
                artist text,
                song text,
                length float,
           PRIMARY KEY (sessionId, itemInSession))
"""

session.execute(query)

In [ ]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
with measure_time():
        file = 'event_datafile_new.csv'

        with open(file, encoding = 'utf8') as f:
            csvreader = csv.reader(f)
            next(csvreader) # skip header
            for line in csvreader:
        ## Assign the INSERT statements into the `query` variable
                query = "INSERT INTO song_info_by_session_and_item (sessionId, itemInSession, artist, song, length)"
                query = query + "VALUES (%s, %s, %s, %s, %s)"
                ## Assign which column element should be assigned for each column in the INSERT statement.
                ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
                session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [ ]:
## Add in the SELECT statement to verify the data was entered into the table
query = "select artist, song, length from song_info_by_session_and_item where sessionId=338 and itemInSession=4"

df = session.execute(query)._current_rows
df

## Query 2:
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userId = 10, sessionId = 182

We don't know if the sessionIds are unique for each user or not. That's why we use the combination of both of them to specify the partition key. 

In [ ]:
query = """CREATE TABLE IF NOT EXISTS song_and_user_info_by_user_and_session (
                userId int,
                sessionId int,
                itemInSession int,
                artist text,
                song text,
                firstName text,
                lastName text,
            PRIMARY KEY ((userId, sessionId), itemInSession));
"""

session.execute(query)

In [ ]:
with measure_time("\n Using BatchStatement: {}s."):
    file = 'event_datafile_new.csv'
    batch = BatchStatement()
    chunksize = 450
    i = 0
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            i+=1
            query = "INSERT INTO song_and_user_info_by_user_and_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
            query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
            batch.add(cassandra.query.SimpleStatement(query), (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
            if(i%chunksize == 0):
                session.execute(batch)
                batch = cassandra.query.BatchStatement()
                i=0
        if(i!=0):
            #process the last chunk, if needed 
            session.execute(batch)

In [ ]:
## SELECT statement to verify the data was entered into the table
query = "select artist, song, firstName, lastName from song_and_user_info_by_user_and_session where userId=10 and sessionId=182"

df = session.execute(query)._current_rows
df

## Query 3: 
## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

We assume that song is enough to be the only partition key and the combination with the userId is not needed. 

If the combination of first and last name were unique, we could remove userId from this table at all and so sort the query result by first or last name.

In [ ]:
query = """CREATE TABLE IF NOT EXISTS user_info_by_song (
                song text,
                userId int,
                firstName text,
                lastName text,
            PRIMARY KEY (song, userId));
"""

session.execute(query)

In [ ]:
with measure_time("\n Using BatchStatement: {}s."):
    file = 'event_datafile_new.csv'
    batch = BatchStatement()
    chunksize = 450
    i = 0
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            i+=1
            query = "INSERT INTO user_info_by_song (song, userId, firstName, lastName)"
            query = query + "VALUES (%s, %s, %s, %s)"
            batch.add(cassandra.query.SimpleStatement(query), (line[9], int(line[10]), line[1], line[4]))
            if(i%chunksize == 0):
                session.execute(batch)
                batch = cassandra.query.BatchStatement()
                i=0
        if(i!=0):
            #process the last chunk, if needed 
            session.execute(batch)

In [ ]:
## SELECT statement to verify the data was entered into the table
query = "select firstName, lastName from user_info_by_song where song='All Hands Against His Own'"

df = session.execute(query)._current_rows
df

### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions

In [ ]:
session.execute('drop table song_info_by_session_and_item')
session.execute('drop table song_and_user_info_by_user_and_session')
session.execute('drop table user_info_by_song')

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()